# Analysis Flow

This notebook demonstrates how an Automate Flow can be used to perform analysis. 
We use a flow that combines Globus Transfer and funcX to move data and then execute an analysis code on the data.

In [1]:
from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)

import time
import json
import sys
import os

from funcx.sdk.client import FuncXClient

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

## Create an analysis function

Use funcX to register a simple analysis function.

In [2]:
fxc = FuncXClient()

In [58]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    from subprocess import PIPE

    os.chdir('/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho')
    python_path = "/home/rchard/.anaconda3/envs/ptycho/bin/python"
    
    
    output_dir = data.get("output_dir", "/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/output/recons/01/")
    pathname = data.get("pathname", "/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/extracted_scan350.h5")
    try:
        os.mkdir(output_dir)
    except:
        pass
    
    cmd = f"{python_path} mpi_tike-recon.py catalyst cgrad 8 --update-positions --input='{pathname}' --folder='{output_dir}'"

    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    return str(res.stdout)
    
func_uuid = fxc.register_function(ptycho)
print(func_uuid)

f373ae25-5beb-418c-baa1-abbd8acbe706


In [53]:
payload = {'pathname': '/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/extracted_scan350.h5',
           'output_dir': '/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/output/recons/06/'}
tutorial_endpoint = '2eb0d751-46f4-4547-be1a-5d7150084623' # Public tutorial endpoint
res = fxc.run(payload, endpoint_id=tutorial_endpoint, function_id=func_uuid)
print(res)

301e4a2d-eb7a-47c3-a79f-71062c464165


In [55]:
fxc.get_result(res)

"/home/rchard/.anaconda3/envs/ptycho/bin/python mpi_tike-recon.py catalyst cgrad 8 --update-positions --input='/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/extracted_scan350.h5' --folder='/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/output/recons/06/'"

## Create the flow

In [56]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 600,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 1800,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint", 
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.dest_path",
            "destination_path.$": "$.input.result_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 600,
      "End": True
    },
  }
}


Register the flow

In [57]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Stills process workflow")
flow_id = flow['id']
print(flow)
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/c929c6da-8882-4f42-b71d-f39a9dac7caf', 'administered_by': [], 'api_version': '1.0', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'Transfer', 'States': {'Analyze': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'Next': 'Transfer2', 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload.$': '$.input'}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 1800}, 'Transfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Initial transfer', 'Next': 'Analyze', 'Parameters': {'destination_endpoint_id.$': '$.input.dest_endpoint', 'source_endpoint_id.$': '$.input.source_endpoint', 'transfer_items': [{'destination_path.$

In [59]:
src_ep = 'ddb59aef-6d04-11e5-ba46-22000b92c6ec' # EP1
dest_ep = 'ddb59af0-6d04-11e5-ba46-22000b92c6ec' # EP2
filename = 'test.txt'

flow_input = {
    "input": {
        "source_endpoint": src_ep,
        "source_path": f"/~/{filename}",
        "dest_endpoint": dest_ep,
        "dest_path": f"/~/{filename}",
        "result_path": f"/~/out_{filename}",
        "fx_id": func_uuid,
        "fx_ep": tutorial_endpoint,
        'pathname': '/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/extracted_scan350.h5',
        'output_dir': '/lus/theta-fs0/projects/ExaLearn/rchard/test_ptycho/output/recons/06/'
    }
}

Run the flow and wait for it to complete

Note: This will try to transfer a file called "test.txt" from your Globus Tutorial Endpoint 1.

In [60]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
print(flow_action)
flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(10)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')

Starting login with Globus Auth, press ^C to cancel.
GlobusHTTPResponse({'action_id': '03d8d694-dc87-458d-8ba2-73980be567cf', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:c4765424-d274-11e5-b894-cb4139f74ecf', 'details': {'code': 'ActionStarted', 'description': 'State Transfer of type Action started', 'details': {'input': {'destination_endpoint_id': 'ddb59af0-6d04-11e5-ba46-22000b92c6ec', 'source_endpoint_id': 'ddb59aef-6d04-11e5-ba46-22000b92c6ec', 'transfer_items': [{'destination_path': '/~/test.txt', 'recursive': False, 'source_path': '/~/test.txt'}]}, 'state_name': 'Transfer', 'state_type': 'Action'}, 'time': '2021-01-29T18:23:06.945000+00:00'}, 'start_time': '2021-01-29T18:23:06.908000+00:00', 'status': 'ACTIVE'})
Flow action started with id: 03d8d694-dc87-458d-8ba2-73980be567cf
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: SUCCEEDED


Check the result of the analysis function

In [61]:
flow_action['details']['output']['AnalyzeResult']

{'action_id': '063df6ab-a369-47d6-a668-4b5b37b6de92',
 'details': {'completion_t': '1611944624.2015426',
  'exception': None,
  'result': "b'[1611944620.646435] [thetagpu13:49012:0]    ucp_context.c:1028 UCX  ERROR exceeded transports/devices limit (70 requested, up to 64 are supported)\\n'",
  'status': 'SUCCEEDED',
  'task_id': '063df6ab-a369-47d6-a668-4b5b37b6de92'},
 'release_after': 'P30D',
 'state_name': 'Analyze',
 'status': 'SUCCEEDED'}